## STATION HOUR:

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv(r'C:\Users\Padmajaa\OneDrive - SSN Trust\IIT KANPUR\ds\station_hour.csv')

C:\Users\Padmajaa\AppData\Local\Temp\ipykernel_17348\2577960979.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'C:\Users\Padmajaa\OneDrive - SSN Trust\IIT KANPUR\ds\station_hour.csv')


In [3]:
df['Datetime'] = pd.to_datetime(df['Datetime'])
df['Month'] = df['Datetime'].dt.month
df['Year'] = df['Datetime'].dt.year
df['Hour'] = df['Datetime'].dt.hour
df['Minute'] = df['Datetime'].dt.minute

In [4]:
print(df.isnull().sum())

StationId           0
Datetime            0
PM2.5          647689
PM10          1119252
NO             553711
NO2            528973
NOx            490808
NH3           1236618
CO             499302
SO2            742737
O3             725973
Benzene        861579
Toluene       1042366
Xylene        2075104
AQI            570190
AQI_Bucket     570190
Month               0
Year                0
Hour                0
Minute              0
dtype: int64


In [5]:
df = df.drop(columns=['NH3', 'Toluene', 'Xylene', 'AQI', 'AQI_Bucket'], axis=1)

In [6]:
columns_to_fill = ['PM2.5','PM10', 'NO', 'NO2', 'NOx', 'CO', 'SO2', 'O3', 'Benzene']
df[columns_to_fill] = df[columns_to_fill].fillna(df[columns_to_fill].mean())

In [7]:
df.head(5)

,StationId,Datetime,PM2.5,PM10,NO,NO2,NOx,CO,SO2,O3,Benzene,Month,Year,Hour,Minute
0,AP001,2017-11-24 17:00:00,60.50,98.00,2.35,30.80,18.25,0.1,11.85,126.40,0.1,11,2017,17,0
1,AP001,2017-11-24 18:00:00,65.50,111.25,2.70,24.20,15.07,0.1,13.17,117.12,0.1,11,2017,18,0
2,AP001,2017-11-24 19:00:00,80.00,132.00,2.10,25.18,15.15,0.1,12.08,98.98,0.2,11,2017,19,0
3,AP001,2017-11-24 20:00:00,81.50,133.25,1.95,16.25,10.23,0.1,10.47,112.20,0.2,11,2017,20,0
4,AP001,2017-11-24 21:00:00,75.25,116.00,1.43,17.48,10.43,0.1,9.12,106.35,0.2,11,2017,21,0


In [8]:
# Define features
time_features = ['Month', 'Year', 'Hour', 'Minute']
air_quality_features = ['PM2.5', 'NO', 'NO2', 'NOx', 'CO', 'SO2', 'O3', 'Benzene']

In [9]:
station_ids = df['StationId'].unique()
station_id_to_index = {station: index for index, station in enumerate(station_ids)}
df['StationIdIndex'] = df['StationId'].map(station_id_to_index)
num_stations = len(station_ids)

In [10]:
df.head(5)

,StationId,Datetime,PM2.5,PM10,NO,NO2,NOx,CO,SO2,O3,Benzene,Month,Year,Hour,Minute,StationIdIndex
0,AP001,2017-11-24 17:00:00,60.50,98.00,2.35,30.80,18.25,0.1,11.85,126.40,0.1,11,2017,17,0,0
1,AP001,2017-11-24 18:00:00,65.50,111.25,2.70,24.20,15.07,0.1,13.17,117.12,0.1,11,2017,18,0,0
2,AP001,2017-11-24 19:00:00,80.00,132.00,2.10,25.18,15.15,0.1,12.08,98.98,0.2,11,2017,19,0,0
3,AP001,2017-11-24 20:00:00,81.50,133.25,1.95,16.25,10.23,0.1,10.47,112.20,0.2,11,2017,20,0,0
4,AP001,2017-11-24 21:00:00,75.25,116.00,1.43,17.48,10.43,0.1,9.12,106.35,0.2,11,2017,21,0,0


In [11]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

def create_sequences(data, station_data, seq_length):
    X, y, X_Station = [], [], []
    
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length, :])  # Sequence of features
        y.append(data[i + seq_length, :len(features)])  # Target: next timestep pollutant values
        X_Station.append(station_data[i + seq_length])  # station index for sequence
        
    return np.array(X), np.array(y), np.array(X_Station)

# Sequence length
seq_length = 30

# Define the features to use for the model
features = ['PM2.5', 'PM10', 'NO', 'NO2', 'NOx', 'CO', 'SO2', 'O3', 'Benzene']

# Extract feature values and other relevant data for sequence generation
data = df[features + ['Month', 'Year', 'Hour', 'Minute']].values
station_data = df['StationIdIndex'].values

# Create sequences using the provided function
X, y, X_Station = create_sequences(data, station_data, seq_length)

# Convert stationIndex to categorical (for multi-station handling)
num_stations = len(df['StationId'].unique())  # Get number of unique cities
X_Station = to_categorical(X_Station, num_classes=num_stations)


In [20]:
def create_sequences(data, station_data, seq_length):
    X, y, X_Station = [], [], []
    
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length, :])  # Sequence of features
        y.append(data[i + seq_length, :len(features)])  # Target: next timestep pollutant values
        X_Station.append(station_data[i + seq_length])  # station index for sequence
        
    return np.array(X), np.array(y), np.array(X_Station)

# Sequence length
seq_length = 30

# Define the features to use for the model
features = ['PM2.5', 'PM10', 'NO', 'NO2', 'NOx', 'CO', 'SO2', 'O3', 'Benzene']

In [21]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Input, Concatenate, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.backend as K
from sklearn.model_selection import train_test_split
import numpy as np

# Define RMSE metric
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

# Create the stacked LSTM model
def create_stacked_lstm_model(seq_length, num_features, num_stations, num_pollutants):
    # Input for the time series sequence data
    input_seq = Input(shape=(seq_length, num_features))  # (Batch, seq_length, num_features)
    input_station = Input(shape=(num_stations,))  # station input as one-hot vector

    # LSTM layers with recurrent dropout
    x = LSTM(128, return_sequences=True, recurrent_dropout=0.1)(input_seq)
    x = LSTM(64, return_sequences=False, recurrent_dropout=0.1)(x)
    x = Dropout(0.3)(x)

    # Batch Normalization
    x = BatchNormalization()(x)

    # Concatenate the output of LSTM with station input
    x = Concatenate()([x, input_station])
    
    # Fully connected layers with regularization
    x = Dense(64, activation='relu', kernel_regularizer=l2(0.001))(x)
    x = Dropout(0.2)(x)
    x = Dense(32, activation='relu', kernel_regularizer=l2(0.001))(x)
    
    # Output layer for pollutant prediction
    output = Dense(num_pollutants, activation='linear')(x)

    # Create and compile the model
    model = Model(inputs=[input_seq, input_station], outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.0005), loss='mse', metrics=[rmse])
    return model

# Data preparation for the model
seq_length = 30  # Sequence length for the LSTM

# Define the features for input
features = ['PM2.5', 'PM10', 'NO', 'NO2', 'NOx', 'CO', 'SO2', 'O3', 'Benzene']
num_features = len(features) + 4  # Adding 4 extra (Month, Year, Hour, Minute)
num_pollutants = len(features)  # Number of pollutants to predict

# Prepare data for sequences
data = df[features + ['Month', 'Year', 'Hour', 'Minute']].values.astype(np.float16)
station_data = df['StationIdIndex'].values.astype(np.int32)

# Create sequences for the model (you need to implement the 'create_sequences' function as before)
X, y, X_station = create_sequences(data, station_data, seq_length)

# Convert stationIndex to one-hot encoding
num_stations = len(df['StationId'].unique())
X_station = to_categorical(X_station, num_classes=num_stations)

# Split data into training and test sets
X_train, X_test, y_train, y_test, X_station_train, X_station_test = train_test_split(
    X, y, X_station, test_size=0.2, random_state=42
)

# Create the model
model = create_stacked_lstm_model(seq_length, num_features, num_stations, num_pollutants)

# Callbacks for early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# Train the model
def data_generator(X, X_station, y, batch_size):
    while True:
        for start in range(0, len(X), batch_size):
            end = min(start + batch_size, len(X))
            yield [X[start:end], X_station[start:end]], y[start:end]

batch_size = 16
train_gen = data_generator(X_train, X_station_train, y_train, batch_size)
val_gen = data_generator(X_test, X_station_test, y_test, batch_size)

history = model.fit(
    train_gen,
    steps_per_epoch=len(X_train) // batch_size,
    validation_data=val_gen,
    validation_steps=len(X_test) // batch_size,
    epochs=100,
    callbacks=[early_stopping, lr_scheduler],
    verbose=1
)

# Predict pollutant concentrations
def predict_pollutants(model, X, X_station):
    predictions = model.predict([X, X_station])
    return predictions

# Example prediction call
predicted_pollutants = predict_pollutants(model, X_test, X_station_test)
print(predicted_pollutants[:5])


MemoryError: Unable to allocate 1.88 GiB for an array with shape (2589053, 30, 13) and data type float16

In [16]:
model.save('C_hr.h5')

In [17]:

from tensorflow.keras.models import load_model
model = load_model('C_hr.h5', custom_objects={'mse': 'mean_squared_error'})

In [19]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model

# Load the saved model with custom metrics
model = load_model('C_hr.h5', custom_objects={'mse': 'mean_squared_error'})

# Function to predict pollutants for a future date and city
def predict_pollutants_for_date(model, future_date, city, seq_length, features, city_to_index):
    # Check if the required historical data is available
    historical_data = df[df['City'] == city].tail(seq_length)  # Get the last 'seq_length' rows for that city
    
    if len(historical_data) < seq_length:
        print(f"Not enough historical data for city: {city} to predict for {future_date}.")
        return
    
    # Prepare the sequence input and city input
    sequence_input = historical_data[features + ['Month', 'Year', 'Hour', 'Minute']].values
    
    # Normalize or reshape the sequence data if necessary
    X_input = np.array([sequence_input])  # Reshape to (1, seq_length, num_features)
    
    # One-hot encode the city index for the city input
    city_index = city_to_index[city]
    X_city_input = np.zeros((1, len(city_to_index)))
    X_city_input[0, city_index] = 1  # One-hot encode the city index
    
    # Make predictions using the loaded model
    predicted_pollutants = model.predict([X_input, X_city_input])

    # Display the predicted concentrations
    print(f"Predicted pollutant concentrations for {city} on {future_date}:")
    pollutants = ['PM2.5', 'PM10', 'NO', 'NO2', 'NOx', 'CO', 'SO2', 'O3', 'Benzene']
    for pollutant, concentration in zip(pollutants, predicted_pollutants[0]):
        print(f"{pollutant}: {concentration:.2f}")

# Example usage
future_date = pd.Timestamp('2025-10-05')
city = 'Chennai'

# Create a mapping from city names to their indices (one-hot encoding)
city_to_index = {city_name: index for index, city_name in enumerate(df['City'].unique())}

# Features used in the prediction
features = ['PM2.5','PM10', 'NO', 'NO2', 'NOx', 'CO', 'SO2', 'O3', 'Benzene']
seq_length = 30  # Adjust according to your model's input requirement

predict_pollutants_for_date(model, future_date, city, seq_length, features, city_to_index)


c:\Users\sunha\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['input_layer_4', 'input_layer_5']. Received: the structure of inputs=('*', '*')
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 708ms/step
Predicted pollutant concentrations for Chennai on 2025-10-05 00:00:00:
PM2.5: 22.75
PM10: 37.74
NO: 5.78
NO2: 14.95
NOx: 14.78
CO: 1.07
SO2: 7.63
O3: 46.62
Benzene: 0.46


In [20]:
import numpy as np
from sklearn.metrics import mean_squared_error, f1_score
from tensorflow.keras.models import load_model
import tensorflow.keras.backend as K

# Define RMSE metric
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

# Define MSE metric
def mse(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true))

# Load the saved model with custom metrics
model = load_model('C_hr.h5', custom_objects={'rmse': rmse, 'mse': mse})

# Get predictions on the test set
predicted_pollutants = model.predict([X_test, X_city_test])

# Binarize predictions and true values based on a threshold (e.g., > 0)
threshold = 0.5  # Adjust threshold as needed
predicted_classes = (predicted_pollutants > threshold).astype(int)
true_classes = (y_test > threshold).astype(int)

# Calculate F1 score
f1_scores = f1_score(true_classes, predicted_classes, average='weighted', zero_division=0)
print(f"F1 Score: {f1_scores}")

# Calculate RMSE for additional evaluation
rmse_value = np.sqrt(mean_squared_error(y_test, predicted_pollutants))
print(f"Root Mean Squared Error (RMSE): {rmse_value}")


4425/4425 ━━━━━━━━━━━━━━━━━━━━ 114s 26ms/step
F1 Score: 0.9636505278760298
Root Mean Squared Error (RMSE): 17.609840010990577
